<a href="https://colab.research.google.com/github/melrahmtz/purple-box/blob/main/hands-on-practice/0503_hybrid_search_v2_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install langchain --quiet
!pip install supabase --quiet
!pip install tiktoken --quiet
!pip install unstructured --quiet
!pip install numpy --quiet
!pip install transformers --quiet
!pip install -U langchain-community  --quiet
!pip install deep-translator --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.5/166.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.0 MB/s eta 0:00:00
   ━━━━━

In [ ]:
import getpass
import os

# Set API keys
if "HUGGINGFACEHUB_API_KEY" not in os.environ:
    os.environ["HUGGINGFACEHUB_API_KEY"] = getpass.getpass("HuggingFace API Key:")

if "SUPABASE_URL" not in os.environ:
    os.environ["SUPABASE_URL"] = getpass.getpass("Supabase URL:")

if "SUPABASE_SERVICE_KEY" not in os.environ:
    os.environ["SUPABASE_SERVICE_KEY"] = getpass.getpass("Supabase Service Key:")


HuggingFace API Key:··········
Supabase URL:··········
Supabase Service Key:··········


# **Trial 1**
Embedding model: `all-MiniLM-L6-v2 (384)`


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from supabase.client import Client, create_client
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize Supabase Client
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# Use MiniLM embedding model (384 dimensions)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    """Generates a 384-dimensional embedding for the input text."""
    return embeddings.embed_query(text)

def load_json_documents(json_path):
    """Loads a JSON file and extracts chunked data."""
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def store_documents_in_supabase(json_path):
    """Stores pre-chunked documents in Supabase without additional chunking."""
    documents = load_json_documents(json_path)

    for node in documents:
        content = node.get("content", "").strip()
        table_data = node.get("table", None)  # Table data if available
        metadata = node.get("metadata", {})

        if not content and not table_data:
            continue  # Skip empty chunks

        embedding = generate_embedding(content) if content else None

        data = {
            "content": content if content else None,
            "embedding": embedding if embedding else None,
            "table_data": json.dumps(table_data) if table_data else None,  # Store tables as JSON
            "source": metadata.get("source", "unknown"),
            "section": metadata.get("section", "unknown"),
            "chunk_position": metadata.get("position", -1)
        }

        supabase.table("documents_hybrid_search_v2").insert(data).execute()

    print(f"Documents from {json_path} successfully stored in Supabase.")


In [ ]:
# Example: Store JSON data
store_documents_in_supabase("17_chunks_v2.json")


Documents from 17_chunks_v2.json successfully stored in Supabase.


In [ ]:
def hybrid_search(query, match_count=10):
    """Calls the hybrid_search function in Supabase with MiniLM embeddings."""
    query_embedding = generate_embedding(query)

    # Call Supabase RPC function
    response = supabase.rpc("hybrid_search_v2", {
        "query_text": query,
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()

    return response.data

# Example query for PDF1
#query = "What is sentiment analysis?"  # Answer in Section 2
#query = "What is the accuracy of the LDA model when using the \"daily weighted average\" sentiment score?"  # Answer in Section 3.1
#query = "What is the characteristic of the sentiment score distribution?"  # Answer in Figure 1

# Example query for 17
query = "What were the objectives of the workshop on family medicine training in Africa?" # Answer in Introduction (page 1)
#query = "Challenges faced in family medicine training in Africa"  # Answer in Introduction (page 2)
#query = "Which countries participated in the workshop on family medicine training in Africa?"  # Answer in Workshop Participant (page 3)
#query = "What factors influence the effectiveness of family medicine training in Africa?" # Answer can be found in several sections

results = hybrid_search(query, match_count=10)

# Display results
for doc in results:
    print(doc)


{'id': 86, 'fts_score': 0.0163934426229508, 'semantic_score': 0.0153846153846154, 'final_score': 0.0317780580075662, 'content': '## Corresponding author:  \nLouis Jenkins, louis.jenkins@westerncape. gov.za\n## Dates:  \nReceived: 28 Sept. 2017  \nAccepted: 09 Nov. 2017  \nPublished: 12 Apr. 2018  \nHow to cite this article: Jenkins LS, Von Pressentin K. Family medicine training in Africa: Views of clinical trainers and trainees. Afr J Prm Health Care Fam Med. 2018;10(1), a1638. https:// doi.org/10.4102/phcfm. v10i1.1638\n## Copyright:  \n© 2018. The Authors. Licensee: AOSIS. This work is licensed under the Creative Commons Attribution License.  \n![Image](/content/markdown/17_artifacts/image_000005_e2cece3be96aa05931eea2488c7312b12d82969056fd50762e3f32ae19090fd2.png)  \n*Image Description:* The image shows a QR code with instructions instructing viewers to "Scan this QR code with their smartphone or mobile device to read online."  \nObjectives :  The  aim  of  the  workshop  was  to  u

In [ ]:
def hybrid_search_display_as_table(query, match_count=10):
    """Calls the hybrid_search function in Supabase and returns results as a DataFrame."""
    query_embedding = generate_embedding(query)

    # Call Supabase RPC function
    response = supabase.rpc("hybrid_search_v2", {
        "query_text": query,
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()

    # Convert results to DataFrame
    if response.data:
        df = pd.DataFrame(response.data, columns=["id", "fts_score", "semantic_score", "final_score", "content", "table_data", "source", "section", "chunk_position"])
        return df
    else:
        print("No results found.")
        return None

# Example query for PDF1
#query = "What is sentiment analysis?"  # Answer in Section 2
#query = "What is the accuracy of the LDA model when using the \"daily weighted average\" sentiment score?"  # Answer in Section 3.1
#query = "What is the characteristic of the sentiment score distribution?"  # Answer in Figure 1

# Example query for 17
#query = "What were the objectives of the workshop on family medicine training in Africa?" # Answer in Introduction (page 1)
#query = "Challenges faced in family medicine training in Africa"  # Answer in Introduction (page 2) or Conclusion
#query = "Which countries participated in the workshop on family medicine training in Africa?"  # Answer in Table Workshop Participant (page 3)
query = "What factors influence the effectiveness of family medicine training in Africa?" # Answer can be found in several sections


df_results = hybrid_search_display_as_table(query, match_count=10)

# Display Table
if df_results is not None:
    from IPython.display import display
    display(df_results)


,id,fts_score,semantic_score,final_score,content,table_data,source,section,chunk_position
0,99,0,0.016393,0.016393,## Competing interests \nThe authors declare ...,None,17.md,Competing interests,16
1,98,0,0.016129,0.016129,## Conclusion \nIt was clear from this worksh...,None,17.md,Conclusion,15
2,87,0,0.015873,0.015873,7 \nThe World Organisation of Family Doctors ...,None,17.md,Unknown,4
3,97,0,0.015625,0.015625,## Reflection on workshop \nThe group appreci...,None,17.md,Reflection on workshop,14
4,90,0,0.015385,0.015385,## Context is critical \nThe physical place w...,None,17.md,Context is critical,7
5,86,0,0.015152,0.015152,"## Corresponding author: \nLouis Jenkins, lou...",None,17.md,Corresponding author:,3
6,91,0,0.014925,0.014925,## Learning style of the registrar and (teachi...,None,17.md,Learning style of the registrar and (teaching ...,8
7,92,0,0.014706,0.014706,## Learning portfolio is utilised \nThe grou...,None,17.md,Learning portfolio is utilised,9
8,85,0,0.014493,0.014493,## Family medicine training in Africa: Views o...,None,17.md,Family medicine training in Africa: Views of c...,2
9,88,0,0.014286,0.014286,## Participants and process \nThirty-five peo...,None,17.md,Participants and process,5


# **Trial 2**
Embedding model: `bge-m3` (1024)

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from supabase.client import Client, create_client
from langchain_community.embeddings import HuggingFaceEmbeddings
from langdetect import detect
from deep_translator import GoogleTranslator

# Initialize Supabase Client
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

# Use bge-m3 multilingual embedding model (1024 dimensions)
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

def translate_text(query_text, target_lang="en"):
    """Detect query language and translate text to English if needed."""
    detected_lang = detect(query_text)

    if detected_lang == "en":
        return query_text, "english"  # No translation needed

    print(f"Detected language: {detected_lang}. Translating to {target_lang}...")

    try:
        translated_text = GoogleTranslator(source="auto", target=target_lang).translate(query_text)
    except Exception as e:
        print(f"Translation failed: {e}")
        translated_text = query_text  # Fallback to original query

    return translated_text, detected_lang  # Return both translated text and detected language

def generate_embedding(text):
    """Generates embedding for the input text."""
    return embeddings.embed_query(text)

def load_json_documents(json_path):
    """Loads a JSON file and extracts chunked data."""
    with open(json_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def store_documents_in_supabase(json_path):
    """Stores pre-chunked documents in Supabase considering both content and table_data."""
    documents = load_json_documents(json_path)

    for node in documents:
        content = node.get("content", "").strip()
        table_data = node.get("table", None)  # Table data if available
        metadata = node.get("metadata", {})

        if not content and not table_data:
            continue  # Skip empty chunks

        text_for_embedding = content if content else json.dumps(table_data)  # Use table_data if content is missing
        embedding = generate_embedding(text_for_embedding)

        data = {
            "content": content if content else None,
            "embedding": embedding if embedding else None,
            "table_data": json.dumps(table_data) if table_data else None,  # Store tables as JSON
            "source": metadata.get("source", "unknown"),
            "section": metadata.get("section", "unknown"),
            "chunk_position": metadata.get("position", -1)
        }

        supabase.table("documents_hybrid_search_v2_bge").insert(data).execute()

    print(f"Documents from {json_path} successfully stored in Supabase.")

<ipython-input-3-b7cdb9ce8902>:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
# Example: Store JSON data
store_documents_in_supabase("Manuale-IRIS_SLIM_IN_TEC_IT_chunks_v2.json")

Documents from Manuale-IRIS_SLIM_IN_TEC_IT_chunks_v2.json successfully stored in Supabase.


In [ ]:
def hybrid_search(query, match_count=10):
    """Calls the hybrid_search function in Supabase with bge-m3 embeddings, after translating the query."""
    translated_query, query_language = translate_text(query)
    query_embedding = generate_embedding(translated_query)

    # Call Supabase RPC function
    response = supabase.rpc("hybrid_search_v2_bge", {
        "query_text": translated_query,
        "query_language": query_language,
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()

    return response.data


# Example query for PDF1
#query = "What is sentiment analysis?"  # Answer in Section 2
#query = "What is the accuracy of the LDA model when using the \"daily weighted average\" sentiment score?"  # Answer in Section 3.1
#query = "What is the characteristic of the sentiment score distribution?"  # Answer in Figure 1

# Example query for 17
#query = "What were the objectives of the workshop on family medicine training in Africa?" # Answer in Introduction (page 1)
#query = "Challenges faced in family medicine training in Africa"  # Answer in Introduction (page 2)
#query = "Which countries participated in the workshop on family medicine training in Africa?"  # Answer in Workshop Participant (page 3)
#query = "What factors influence the effectiveness of family medicine training in Africa?" # Answer can be found in several sections

# Example query for Manuale-IRIS_SLIM_IN_TEC_IT
query = "What are the key considerations for using and maintaining the Iris Slim units?"  # this question could be answer from different paragraphs (Section 2 and Section 6.1)

results = hybrid_search(query, match_count=10)

# Display results
for doc in results:
    print(doc)


{'id': 62, 'fts_score': 0, 'semantic_score': 0.0163934426229508, 'final_score': 0.0163934426229508, 'content': "## 2. Presentazione\n## 2.1. Uso Previsto  \nLe  unità  Iris  Slim  sono  progettate  per  la  funzione  di  riscaldamento,  raffrescamento,  deumidificazione  e  filtrazione  di ambienti residenziali e terziario (uffici, locali pubblici, o simili).\n## 2.2. Usi Non Previsti E Controindicazioni  \nNon sono ammesse le seguenti applicazioni:  \n- · Funzionamento all'aperto\n- · Funzionamento in ambienti umidi o esplosivi o polverosi\n- · Funzionamento in ambienti corrosivi, in particolare per le alette d'alluminio della batteria\n- · Funzionamento in ambienti sottoposti a disturbi elettromagnetici  \nL'uso delle macchine non è destinato a persone (bambini compresi) con ridotte capacità fisiche, mentali o sensoriali, o a persone che non hanno ricevuto un'adeguata informazione, a meno che ciò avvenga sotto la supervisione di una persona responsabile della loro sicurezza", 'table_

In [ ]:
def hybrid_search_display_as_table(query, match_count=10):
    """Calls the hybrid_search function in Supabase with bge-m3 embeddings, after translating the query."""
    translated_query, query_language = translate_text(query)
    query_embedding = generate_embedding(translated_query)

    # Call Supabase RPC function
    response = supabase.rpc("hybrid_search_v2_bge", {
        "query_text": translated_query,
        "query_language": query_language,
        "query_embedding": query_embedding,
        "match_count": match_count
    }).execute()

    # Convert results to DataFrame
    if response.data:
        df = pd.DataFrame(response.data, columns=["id", "fts_score", "semantic_score", "final_score", "content", "table_data", "source", "section", "chunk_position"])
        return df
    else:
        print("No results found.")
        return None

# Example query for PDF1
#query = "What is sentiment analysis?"  # Answer in Section 2
#query = "What is the accuracy of the LDA model when using the \"daily weighted average\" sentiment score?"  # Answer in Section 3.1
#query = "What is the characteristic of the sentiment score distribution?"  # Answer in Figure 1

# Example query for 17
#query = "What were the objectives of the workshop on family medicine training in Africa?" # Answer in Introduction (page 1)
#query = "Challenges faced in family medicine training in Africa"  # Answer in Introduction (page 2)
#query = "Which countries participated in the workshop on family medicine training in Africa?"  # Answer in Workshop Participant (page 3)
#query = "What factors influence the effectiveness of family medicine training in Africa?" # Answer can be found in several sections

# Example query for Manuale-IRIS_SLIM_IN_TEC_IT
#query = "What are the key considerations for using and maintaining the Iris Slim units?"  # Answer in Section 2 and Section 6.1
#query = "What steps should be taken in case of water leakage from the IRIS Slim unit?"  # Answer in Section 6, Section 6.3.1, Section 4.3.1
#query = "What are the operating limits of the IRIS Slim unit?"  # Answer in Section 2.5 (Table)
query = "What are the installation requirements for the IRIS Slim unit?"  # Answer in Section 4.2

df_results = hybrid_search_display_as_table(query, match_count=10)

# Display Table
if df_results is not None:
    from IPython.display import display
    display(df_results)


,id,fts_score,semantic_score,final_score,content,table_data,source,section,chunk_position
0,62,0,0.016393,0.016393,## 2. Presentazione\n## 2.1. Uso Previsto \nL...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,2. Presentazione,7
1,73,0,0.016129,0.016129,## 4.2. Posizionamento \nL'unità deve essere ...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,4.2. Posizionamento,18
2,57,0,0.015873,0.015873,None,"{""headers"": [""1. Introduzione ...................",Manuale-IRIS_SLIM_IN_TEC_IT.md,Table,2
3,56,0,0.015625,0.015625,![Image](/content/markdown/Manuale-IRIS_SLIM_I...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,Unknown,1
4,72,0,0.015385,0.015385,None,"{""headers"": ["""", """", ""551"", ""751"", """"], ""rows""...",Manuale-IRIS_SLIM_IN_TEC_IT.md,Table,17
5,77,0,0.015152,0.015152,## Schemi Elettrici IRIS SLIM \nPer le tavole...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,Schemi Elettrici IRIS SLIM,22
6,74,0,0.014925,0.014925,## 4.3. Alimentazioni\n## 4.3.1. Connessioni I...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,4.3. Alimentazioni,19
7,76,0,0.014706,0.014706,## 4.3.2. Connessioni Elettriche \nPrevedere ...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,4.3.2. Connessioni Elettriche,21
8,60,0,0.014493,0.014493,## NOTA \nNOTE: \n![Image](/content/markdown...,None,Manuale-IRIS_SLIM_IN_TEC_IT.md,NOTA,5
9,69,0,0.014286,0.014286,"## 3. Sollevamento, Disimballo E Controllo Del...",None,Manuale-IRIS_SLIM_IN_TEC_IT.md,"3. Sollevamento, Disimballo E Controllo Del Co...",14
